Face recognition by using transfer learning and VGG16 model

https://www.kaggle.com/dansbecker/5-celebrity-faces-dataset

This dataset has 5 celebrity:
- elton_john
- ben_afflek
- jerry_seinfeld
- madonna
- mindy_kaling


In [ ]:
import numpy as np
from glob import glob
import tensorflow as tf
from tensorflow.keras.preprocessing import image 

from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Lambda, Dense, Flatten
from sklearn.metrics import classification_report, confusion_matrix
from keras import metrics
from tensorflow.keras.layers import Conv2D, Dense
from keras.regularizers import l2

In [ ]:
IMAGE_SIZE = [160,160]

training_folder = 'train/'
validating_folder = 'val/'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(training_folder, target_size = IMAGE_SIZE,
                                                 batch_size = 8,
                                                 class_mode = "categorical")

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
test_set = test_datagen.flow_from_directory(validating_folder, target_size = IMAGE_SIZE,
                                                 batch_size = 8,
                                                 class_mode = "categorical",
                                                 shuffle = False)

In [ ]:
vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
prediction = Dense(5, activation = 'softmax')(x)

In [ ]:
model = Model(inputs = vgg.input, outputs = prediction)

In [ ]:
model.summary()

In [ ]:
training_set.shuffle= True

In [ ]:
model.compile(optimizer= 'adam', loss= 'CategoricalCrossentropy', metrics = ['accuracy'])

r = model.fit(x = training_set, validation_data = test_set, epochs = 5)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label = 'train loss')
plt.plot(r.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['accuracy'], label = 'train acc')
plt.plot(r.history['val_accuracy'], label = 'val acc')
plt.legend()
plt.show()

In [ ]:
def recognize_face(filename):
    test_image = image.load_img(filename, target_size = (160,160))
    test_image = image.img_to_array(test_image)
    test_image = test_image/255
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    return result

In [ ]:
np.set_printoptions(suppress=True)
result = recognize_face('val/mindy_kaling/4.jpg')
print(result)
np.argmax(result)


In [ ]:
np.argmax(model.predict(test_set),axis=1)

### Zbiór uczacy - macierz pomyłek

In [ ]:
training_set.shuffle= False

In [ ]:
y_pred = model.predict(training_set)
predicted_classes = np.argmax(y_pred, axis=1)
true_classes = training_set.classes
class_labels = list(training_set.class_indices.keys())   
report = confusion_matrix(true_classes, predicted_classes)
print(report) 

### Zbiór testowy - macierz pomyłek

In [ ]:
y_pred = model.predict(test_set)
predicted_classes = np.argmax(y_pred, axis=1)
true_classes = test_set.classes
class_labels = list(test_set.class_indices.keys())   

In [ ]:
report = confusion_matrix(true_classes, predicted_classes)
print(report)    

In [ ]:
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)    